# **E1. ACTIVIDAD INTEGRADORA 1**

Rorigo Lopez Murguia

Victor Misael Escalante Alvarado



## Dependencias del proyecto

Funciones utilizadas para el funcionamiento del proyecto (Es necesario ejecutar antes de las siguientes celdas)

In [ ]:
#Funciones del proyecto

class Protein:
    def __init__(self, name, sequence):
        self.name = name
        self.sequence = sequence

    def __str__(self):
        return f"{self.name}: {self.sequence}"

#Z array para string-matching
def ZArray(texto, patron):
    # Z - Array Algorithm
    n = len(texto)
    m = len(patron)

    L = 0
    R = 0
    x = n + m + 1
    Z = [-1 for i in range(x)]
    C = patron + "$" + texto

    for i in range(x):
        if i <= R:
            k = i - L
            if Z[k] < R - i + 1:
                Z[i] = Z[k]
            else:
                L = i
                while R < x and C[R-L] == C[R]:
                    R += 1
                Z[i] = R - L
                R -= 1
        else:
            L = R = i
            while R < x and C[R-L] == C[R]:
                R += 1
            Z[i] = R - L
            R -= 1
    return Z

def StringMatchZ(texto, patron):
  m = len(texto)
  n = len(patron)
  m = m + n + 1
  indexes = []
  zArray = ZArray(texto,patron)
  for i in range(m):
    if zArray[i] == n:
      # return i - n - 1
      indexes.append(i-n-1)
  return indexes

#Manacher para palíndromo más largo en los genes
def Manacher(frase):
    centro = limite = 0

    texto = "@$" + "$".join(frase) + "$#"
    e = len(texto)

    P = [0] * e

    for i in range(1, e - 1):
        if i < limite:
            simetrica = 2 * centro - i
            P[i] = min(limite - i, P[simetrica])

        while (i + P[i] + 1 < e and
               i - P[i] - 1 >= 0 and
               texto[i + P[i] + 1] == texto[i - P[i] - 1]):
            P[i] += 1

        if i + P[i] > limite:
            limite = i + P[i]
            centro = i

    maxVal = max(P)
    maxIndex = P.index(maxVal)
    inicio = (maxIndex - maxVal) // 2
    fin = inicio + maxVal
    return frase[inicio:fin], inicio, fin

#Trie Trees para búsqeuda de aminoácidos en las proteinas
class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.isFinal = False
        self.aminoacid = None
        self.hijos = []

    def agregar_hijo(self, valor):
        nuevo_nodo = Nodo(valor)
        self.hijos.append(nuevo_nodo)
        return nuevo_nodo

    def search_hijo(self, valor):
        for hijo in self.hijos:
            if hijo.valor == valor:
                return hijo
        return None

    def __str__(self):
        return str(self.valor)

class Trie:
    def __init__(self):
        self.raiz = Nodo("#")

    def agregar_palabra(self, palabra, aminoacid):
        actual = self.raiz
        for letra in palabra:
            hijo = actual.search_hijo(letra)
            if hijo is None:
                hijo = actual.agregar_hijo(letra)
            actual = hijo
        actual.isFinal = True
        actual.aminoacid = aminoacid

    def mostrar_arbol(self):
        actual = self.raiz
        queue = deque([actual])

        while queue:
            for _ in range(len(queue)):
                nodo = queue.popleft()
                print(nodo, end=" ")
                for hijo in nodo.hijos:
                    queue.append(hijo)
            print()

    def buscar_palabra(self, palabra):
        actual = self.raiz
        for letra in palabra:
            hijo = actual.search_hijo(letra)
            if hijo is None:
                return None
            else:
                actual = hijo
        if actual.isFinal:
            return actual.aminoacid
        else:
            return "-"

def buscar_palabras(nodo, palabra):
    palabras = []
    if nodo.isFinal:
        palabras.append(palabra)
    for hijo in nodo.hijos:
        palabras += buscar_palabras(hijo, palabra + hijo.valor)
    return palabras

# Buscar proteinas en secuencia
def SearchProteinas(sequence):
  lista_aminoacidos = []
  reading_frames = [0,1,2]

  # Por cada inicio de proteina, traducir hasta encontrar un STOP o hasta que acabe
  while len(reading_frames) > 0:
      start_index = reading_frames[0]
      end_index = reading_frames[0]

      protein = ""
      for i in range(start_index, len(sequence), 3): # Itera de 3 en 3 para buscar los codones
        codon = sequence[i:i+3]
        if len(codon) == 3:
          aminoacid = diccionario_aminoacidos.buscar_palabra(codon)
          protein += aminoacid
          end_index = i+3

      lista_aminoacidos.append((reading_frames[0], protein))
      reading_frames.pop(0)

  results = []

  # Buscar las proteinas en las listas encontradas
  # for aminoacid in lista_aminoacidos:
  #    for protein in proteins:
  for protein in proteins:
    encontrado = False
    for aminoacid in lista_aminoacidos:
      z = StringMatchZ(aminoacid[1], protein.sequence)
      if len(z) > 0:
        encontrado = True
        results.append((protein.name, z[0]*3, protein.sequence[0:4], sequence[z[0]*3:z[0]*3+12]))
    if not encontrado:
      results.append((protein.name, "No encontrado", "", ""))
  return results

# Operaciones para el punto 4

def get_codon(sequence, index):
  return (diccionario_aminoacidos.buscar_palabra(sequence[index:index+3]), sequence[index:index+3])

def get_diff(sequence1, sequence2):
  results = []
  reading_frames = [0,1]

  # Por cada inicio de proteina, traducir hasta encontrar un STOP o hasta que acabe
  while len(reading_frames) > 0:
      start_index = reading_frames[0]
      end_index = reading_frames[0]

      protein = ""
      for i in range(start_index, min(len(sequence1), len(sequence2)), 3): # Itera de 3 en 3 para buscar los codones
        # codon1 = sequence1[i:i+3]
        # codon2 = sequence2[i:i+3]
        codon1, aminoacid = get_codon(sequence1, i)
        codon2, aminoacid2 = get_codon(sequence2, i)
        if codon1 != codon2:
          aparece = False
          for result in results:
            if abs(result[0] - i) <= 20:
              aparece = True
              break
          if not aparece:
            results.append((i, (aminoacid, codon1), (aminoacid2, codon2)))

      reading_frames.pop(0)

  return results

#Creación de diccionario Trie para búsqueda de aminoácidos

aminoacids = [
    # Alanine
    ("GCT", "A"), ("GCC", "A"), ("GCA", "A"), ("GCG", "A"),
    # Arginine
    ("CGT", "R"), ("CGC", "R"), ("CGA", "R"), ("CGG", "R"), ("AGA", "R"), ("AGG", "R"),
    # Asparagine
    ("AAT", "N"), ("AAC", "N"),
    # Aspartic Acid
    ("GAT", "D"), ("GAC", "D"),
    # Cysteine
    ("TGT", "C"), ("TGC", "C"),
    # Glutamic Acid
    ("GAA", "E"), ("GAG", "E"),
    # Glutamine
    ("CAA", "Q"), ("CAG", "Q"),
    # Glycine
    ("GGT", "G"), ("GGC", "G"), ("GGA", "G"), ("GGG", "G"),
    # Histidine
    ("CAT", "H"), ("CAC", "H"),
    # Isoleucine
    ("ATT", "I"), ("ATC", "I"), ("ATA", "I"),
    # Leucine
    ("CTT", "L"), ("CTC", "L"), ("CTA", "L"), ("CTG", "L"), ("TTA", "L"), ("TTG", "L"),
    # Lysine
    ("AAA", "K"), ("AAG", "K"),
    # Methionine (Start Codon)
    ("ATG", "M"),
    # Phenylalanine
    ("TTT", "F"), ("TTC", "F"),
    # Proline
    ("CCT", "P"), ("CCC", "P"), ("CCA", "P"), ("CCG", "P"),
    # Serine
    ("TCT", "S"), ("TCC", "S"), ("TCA", "S"), ("TCG", "S"), ("AGT", "S"), ("AGC", "S"),
    # Threonine
    ("ACT", "T"), ("ACC", "T"), ("ACA", "T"), ("ACG", "T"),
    # Tryptophan
    ("TGG", "W"),
    # Tyrosine
    ("TAT", "Y"), ("TAC", "Y"),
    # Valine
    ("GTT", "V"), ("GTC", "V"), ("GTA", "V"), ("GTG", "V"),
    # Stop Codons
    ("TAA", "*"), ("TAG", "*"), ("TGA", "*")
]

amino_acid_dict = {
    'A': 'Alanine',
    'R': 'Arginine',
    'N': 'Asparagine',
    'D': 'Aspartic Acid',
    'C': 'Cysteine',
    'E': 'Glutamic Acid',
    'Q': 'Glutamine',
    'G': 'Glycine',
    'H': 'Histidine',
    'I': 'Isoleucine',
    'L': 'Leucine',
    'K': 'Lysine',
    'M': 'Methionine',
    'F': 'Phenylalanine',
    'P': 'Proline',
    'S': 'Serine',
    'T': 'Threonine',
    'W': 'Tryptophan',
    'Y': 'Tyrosine',
    'V': 'Valine',
    '*': 'Stop Codon'
}


diccionario_aminoacidos = Trie()

for aminoacido in aminoacids:
    diccionario_aminoacidos.agregar_palabra(aminoacido[0], aminoacido[1])
    print(aminoacido)


('GCT', 'A')
('GCC', 'A')
('GCA', 'A')
('GCG', 'A')
('CGT', 'R')
('CGC', 'R')
('CGA', 'R')
('CGG', 'R')
('AGA', 'R')
('AGG', 'R')
('AAT', 'N')
('AAC', 'N')
('GAT', 'D')
('GAC', 'D')
('TGT', 'C')
('TGC', 'C')
('GAA', 'E')
('GAG', 'E')
('CAA', 'Q')
('CAG', 'Q')
('GGT', 'G')
('GGC', 'G')
('GGA', 'G')
('GGG', 'G')
('CAT', 'H')
('CAC', 'H')
('ATT', 'I')
('ATC', 'I')
('ATA', 'I')
('CTT', 'L')
('CTC', 'L')
('CTA', 'L')
('CTG', 'L')
('TTA', 'L')
('TTG', 'L')
('AAA', 'K')
('AAG', 'K')
('ATG', 'M')
('TTT', 'F')
('TTC', 'F')
('CCT', 'P')
('CCC', 'P')
('CCA', 'P')
('CCG', 'P')
('TCT', 'S')
('TCC', 'S')
('TCA', 'S')
('TCG', 'S')
('AGT', 'S')
('AGC', 'S')
('ACT', 'T')
('ACC', 'T')
('ACA', 'T')
('ACG', 'T')
('TGG', 'W')
('TAT', 'Y')
('TAC', 'Y')
('GTT', 'V')
('GTC', 'V')
('GTA', 'V')
('GTG', 'V')
('TAA', '*')
('TAG', '*')
('TGA', '*')


# Lectura de archivos

In [ ]:
# Leer archivos

with open('SARS-COV-2-MN908947.3.txt', 'r') as file:
    lines = file.readlines()
    secuencia_wuhan = ""
    for line in lines[1:]:
        line = line.strip()
        secuencia_wuhan += line

with open('SARS-COV-2-MT106054.1.txt', 'r') as file:
    lines = file.readlines()
    secuencia_texas = ""
    for line in lines[1:]:
        line = line.strip()
        secuencia_texas += line

with open('gen-M.txt', 'r') as file:
    lines = file.readlines()
    gen_m = ""
    for line in lines[1:]:
        line = line.strip()
        gen_m += line

with open('gen-S.txt', 'r') as file:
    lines = file.readlines()
    gen_s = ""
    for line in lines[1:]:
        line = line.strip()
        gen_s += line

with open('gen-ORF1AB.txt', 'r') as file:
    lines = file.readlines()
    gen_o = ""
    for line in lines[1:]:
        line = line.strip()
        gen_o += line

with open('seq-proteins.txt', 'r') as file:
    lines = file.readlines()
    proteins = []
    for line in lines:
        if line.startswith(">"):
            name = line.strip()[1:]
            sequence = ""
        else:
            sequence += line.strip()
            proteins.append(Protein(name, sequence))

# 1. Aparicion de Los genes

En la siguiente seccion se obtienen los **indices** de aparicion de los genes en la secuencia del virus *SARS-COV-2-MN908947.3*

**Complejidades**

Para la búsqueda de las cadenas de los genes en la cadena de Wuhan (2019) se utilizó string matching con Z-Array, que tiene una complejidad de O(n), donde en este caso, n es la cantidad de aminoácidos en la secuencia. El procedimiento se realiza una vez por cada gen a buscar.

In [ ]:
# Indices de aparición de los genes en la secuencia del virus SARS-COV-2-MN908947.3.txt

print("Apariciones del gen M: ")
gen_m_indices = StringMatchZ(secuencia_wuhan, gen_m)
print(gen_m_indices)
print(secuencia_wuhan[gen_m_indices[0]:gen_m_indices[0]+12])

print("Apariciones del gen S: ")
gen_s_indices = StringMatchZ(secuencia_wuhan, gen_s)
print(gen_s_indices)
print(secuencia_wuhan[gen_s_indices[0]:gen_s_indices[0]+12])

print("Apariciones del gen ORF1AB: ")
gen_o_indices = StringMatchZ(secuencia_wuhan, gen_o)
print(gen_o_indices)
print(secuencia_wuhan[gen_o_indices[0]:gen_o_indices[0]+12])

Apariciones del gen M: 
[26522]
ATGGCAGATTCC
Apariciones del gen S: 
[21562]
ATGTTTGTTTTT
Apariciones del gen ORF1AB: 
[265]
ATGGAGAGCCTT


# 2. Palindromo Mas largo

En la siguiente seccion se muestran los palindromos mas largos para cada uno de los genomas


---

**Complejidades**
Para encontrar palíndromos, se utilizó el algorítmo de Manacher, que cuenta con una complejidad O(n) donde n es la cantidad de aminoácidos en la secuencia.

In [ ]:
#Palíndromo más largo en cada uno de los tres genes

print("Palíndromo más grande para gen M")
palindrome_m, inicio_m, fin_m = Manacher(gen_m)
print(palindrome_m, "\tIndice:", inicio_m, "\tLongitud:", len(palindrome_m))
print('\n')

print("Palíndromo más grande para gen S")
palindrome_s, inicio_s, fin_s = Manacher(gen_s)
print(palindrome_s, "\tIndice:", inicio_s, "\tLongitud:", len(palindrome_s))
print('\n')

print("Palíndromo más grande para gen ORF1AB")
palindrome_o, inicio_o, fin_o = Manacher(gen_o)
print(palindrome_o, "\tIndice:", inicio_o, "\tLongitud:", len(palindrome_o))
print('\n')

# Escribir palindromos en archivo
with open('palindromos.txt', 'w') as file:
  file.write("M\n"+str(inicio_m)+","+str(fin_m)+","+palindrome_m+"\n")
  file.write("S\n"+str(inicio_s)+","+str(fin_s)+","+palindrome_s+"\n")
  file.write("ORF1AB\n"+str(inicio_o)+","+str(fin_o)+","+palindrome_o+"\n")
  file.close()

Palíndromo más grande para gen M
CTAAAGAAATC 	Indice: 493 	Longitud: 11


Palíndromo más grande para gen S
GAATTTGTGTTTAAG 	Indice: 570 	Longitud: 15


Palíndromo más grande para gen ORF1AB
CTCAATGACTTCAGTAACTC 	Indice: 9711 	Longitud: 20




# 3. Busqueda de Proteinas

En este apartado mostramos las apariciones de las proteinas de "seq-proteins.txt" en la secuencia del virus de de Wuhan (2019)

Se muestra el nombre de la proteína, el índice donde se comienza a producir, sus primeros aminoácidos y sus codones asociados

---

**Estructuras**

Para relacionar trios de aminoácidos con codones, se utilizó un diccionario Trie que ayuda a almacenar y encontrar las diferentes posibilidades de codones en una sola estructura.

**Procedimiento**

Se encontraron los posibles reading frames para la lectura y traducción. A partir de ahí, se leen los codones de 3 en 3 aminoácidos hasta acabar la secuencia. Una vez traducidos los reading frames, se buscan las proteínas en la secuencia generada con String Matching usando un Z array.

**Complejidad**

 El procedimiento completo tiene una complejidad en el dominio de O(n), donde n es la cantidad de aminoácidos en la secuencia.

In [ ]:
res = SearchProteinas(secuencia_wuhan)
for re in res:
  if re[1] != "No encontrado":
    print(re[0], ":", re[1], "- 4 amino:", re[2], "=", re[3])
  else:
    print(re[0], ":", re[1])

QHD43415_1 : 264 - 4 amino: MESL = GATGGAGAGCCT
QHD43415_2 : 804 - 4 amino: AYTR = GGCATACACTCG
QHD43415_3 : 2718 - 4 amino: APTK = TGCACCAACAAA
QHD43415_4 : 8553 - 4 amino: KIVN = TAAAATTGTTAA
QHD43415_5 : 10053 - 4 amino: SGFR = GAGTGGTTTTAG
QHD43415_6 : 10971 - 4 amino: SAVK = AAGTGCAGTGAA
QHD43415_7 : 11841 - 4 amino: SKMS = GTCTAAAATGTC
QHD43415_8 : 12090 - 4 amino: AIAS = AGCTATAGCCTC
QHD43415_9 : 12684 - 4 amino: NNEL = GAATAATGAGCT
QHD43415_10 : 13023 - 4 amino: AGNA = AGCTGGTAATGC
QHD43415_11 : No encontrado
QHD43415_12 : 16236 - 4 amino: AVGA = GCTGTTGGGGCT
QHD43415_13 : 18039 - 4 amino: AENV = GCTGAAAATGTA
QHD43415_14 : 19620 - 4 amino: SLEN = AGTTTAGAAAAT
QHD43415_15 : 20658 - 4 amino: SSQA = TCTAGTCAAGCG
QHD43416 : 21561 - 4 amino: MFVF = AATGTTTGTTTT
QHD43417 : 25392 - 4 amino: MDLF = ATGGATTTGTTT
QHD43418 : 26244 - 4 amino: MYSF = ATGTACTCATTC
QHD43419 : 26520 - 4 amino: MADS = CCATGGCAGATT
QHD43420 : 27201 - 4 amino: MFHL = ATGTTTCATCTC
QHD43421 : 27393 - 4 amino: MKII 

# 4. Comparar proteinas entre Wuhan (2019) y Texas (2020)

Para este punto, similar al punto 3, se calcularon y se tradujeron los reading frames para realizar comparación entre ambas secuencias desde las diferentes perspectivas de los reading frames, respuestas repetidas no se tomaron en cuenta.

**Complejidad**

Se recorren 3 veces las secuencias, por lo que el algoritmo tiene una complejidad en el dominio de O(n), donde n es la cantidad de aminoácidos en la secuencia.

In [ ]:
res = get_diff(secuencia_wuhan, secuencia_texas)
print("Indice\t\tWuhan\t\tTexas")
for r in res:
  # print(r)
  # print("Diferencia en", r[0], ":", r[1][0], "("+r[1][1]+")", "vs", r[2][0], "("+r[2][1]+")")
  print(str(r[0]) + "\t\t" + str(r[1][0])+ " (" + str(r[1][1]) + ")" + "\t\t" + str(r[2][0])+ " (" + str(r[2][1]) + ")")

Indice		Wuhan		Texas
8781		CCA (P)		TCA (S)
19173		GAT (D)		GCT (A)
27924		CAA (Q)		TAA (*)
28143		TAC (Y)		CAC (H)
29094		CGG (R)		TGG (W)
29880		AAA (K)		AA (-)
18601		ATG (M)		ACG (T)
18973		TTC (F)		TAC (Y)
